In [1]:
# Data packages
import pandas as pd 
import os

import pandas as pd
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from sklearn.model_selection import train_test_split

from model.rnn import GRUDecoder
from model.autoencoder import AutoEncoder, SimpleAutoEncoder
from model.layers import ConvEncoderLayer, ConvDecoderLayer, Norm
from data_processing.pipeline import encoding_pipeline, get_generic_name

import torch.nn as nn
import torch.nn.functional as F
from torch_frame.nn.conv import TabTransformerConv, ExcelFormerConv
from torch_frame.nn.decoder import ExcelFormerDecoder
from model.embeddings import SurveyEmbeddings

In [2]:
def get_device():
    # Check if CUDA is available
    if torch.cuda.is_available():
        # If CUDA is available, select the first CUDA device
        device = torch.device("cuda:0")
        print("Using CUDA device:", torch.cuda.get_device_name(0))
    # Check for MPS availability on supported macOS devices (requires PyTorch 1.12 or newer)
    elif torch.backends.mps.is_available():
        # If MPS is available, use MPS device
        device = torch.device("mps")
        print("Using MPS (Metal Performance Shaders) device")
    else:
        # Fallback to CPU if neither CUDA nor MPS is available
        device = torch.device("cpu")
        print("Using CPU")
    return device
device = get_device()

Using MPS (Metal Performance Shaders) device


# Read the data

Right now the notebook is set to work with fake data. This can be changed once the pipeline works.

The data is stored as a Dict[person_id, Sequences] where Sequences is a Dict[year, survery_wave_response]

In [3]:
# read in data and prepare transformations
data = pd.read_csv("data/training_data/PreFer_train_data.csv")
targets = pd.read_csv('data/training_data/PreFer_train_outcome.csv')
codebook = pd.read_csv('data/codebooks/PreFer_codebook.csv')

/var/folders/pk/3vzybg253k1d3n7qzxkts_2c0000gn/T/ipykernel_11838/1529815470.py:2: DtypeWarning: Columns (2583,2584,2585,2586,2587,2588,2589,4358,4359,4360,4361,4362,4363,4364,4365,4366,4367,4368,4369,4370,4371,4372,4373,4374,4375,4379,4380,4381,4382,4383,4384,4385,4386,4387,4388,4389,4390,4391,4392,4393,4394,4395,4396,4397,4398,4399,4400,4401,4405,4406,4407,4408,4409,5215,5216,5219,5220,5613,5614,5615,5616,5617,5618,5619,5620,5621,5622,5624,5625,5626,5627,5628,5629,5630,5631,5632,5633,5634,5635,5636,5638,5639,5640,5787,5788,5789,5790,5791,5792,5793,5794,5795,5796,6393,6394,6395,6396,6397,6398,6399,6400,6401,6402,6403,6619,6620,6621,6622,6623,6624,6625,6626,6627,6628,6629,6630,6631,6632,6633,6634,6635,6638,6640,6641,6642,6643,6644,6645,6646,6647,6648,6649,6650,6651,6652,6653,6654,6655,6656,6657,6658,6659,6660,6661,6664,6666,6667,6668,6669,6670,6965,6966,6967,6968,6969,6970,6971,6972,6973,6974,6975,7064,7065,7066,7067,7068,7069,7070,7071,7072,7073,7074,7163,7164,7165,7166,7167,7168,7169,

In [4]:
importance = pd.read_csv('features_importance_1000.csv')
custom_pairs = importance.iloc[:50].feature.map(lambda x: get_generic_name(x))

In [5]:
# check if sequences have been preprocessed (saves time)
if False:# os.path.exists('data/processed_data/sequences.pt'):
    sequences = torch.load('data/processed_data/sequences.pt')
else:
    sequences = encoding_pipeline(data, codebook, custom_pairs=custom_pairs)
    #torch.save(sequences, 'data/processed_data/sequences.pt')

/Users/carlomarx/Documents/GitHub/fertility-prediction-challenge/data_processing/pipeline.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  codebook["pairs"] = codebook['var_name'].apply(get_generic_name)


In [6]:
from model.dataset import PretrainingDataset
pretrain_dataset = PretrainingDataset(sequences)

# Experimental Encoder (Only)

In [8]:


class ExpEncoder(nn.Module):
    def __init__(self, vocab_size, sequence_len: int, embedding_size: int, output_size: int, num_cols: int, num_layers: int = 3) -> None:
        super().__init__()

        self.embedding = SurveyEmbeddings(
            vocab_size, sequence_len, n_years=14, embedding_dim=embedding_size)

        self.encoders = nn.ModuleList([
            nn.Sequential(
                ExcelFormerConv(channels=embedding_size, num_cols=num_cols, num_heads=4),
                nn.LayerNorm(embedding_size))
                 for _ in range(num_layers)])
        
        self.flatten = ExcelFormerDecoder(in_channels = embedding_size, 
                                          out_channels=output_size, 
                                          num_cols= num_cols)
        

    def forward(self, year, seq):
        """
        Method that returns full encoding-decoding
        """
        x = self.embedding(year, seq)
        for encoder in self.encoders:
            x = encoder(x)
        x = self.flatten(x)
        return x


# Train the RNN

First we need to create Dataset class that can hold both the target (stored in a pd.DataFrame) and the sequences.

The sequences will be of dimension 14 x encoding_dimension, because we have 14 years of surveys.

I have created some code for getting the data into the right format, but it might not be useful.

## Regarding masks
Right now the masking is done already in the encoding. I haven't found exactly where Mikkel implemented this.
So for now, assume that nothing is padded, and then we'll figure it out with Mikkel.

In [9]:
# its not everyone we have a target for, so we do restrict the data to 
# the ones with known outcomes
targets = targets[targets.new_child.notna()]
train_person_ids, test_person_ids = train_test_split(targets['nomem_encr'], test_size=0.2, random_state=42)

In [10]:
rnn_data = {person_id: (
        torch.tensor([year-2007 for year, _ in wave_responses.items()]).to(device),
        torch.tensor([ wave_response for _, wave_response in wave_responses.items()]).to(device)
        )
        for person_id, wave_responses in sequences.items()
}

In [11]:
# split data based on the splits made for the target
train_data = {person_id: rnn_data[person_id] for person_id in train_person_ids}
test_data = {person_id: rnn_data[person_id] for person_id in test_person_ids}

In [12]:
from model.dataset import FinetuningDataset
train_dataset = FinetuningDataset(train_data, targets = targets)
test_dataset = FinetuningDataset(test_data, targets = targets)

rnn_batch_size = 4

train_dataloader = DataLoader(train_dataset, batch_size=rnn_batch_size, shuffle=True)
test_dataloader  = DataLoader(test_dataset,  batch_size=rnn_batch_size)

In [20]:
# ft - fine-tuning

HIDDEN_SIZE = 128
ENCODING_SIZE = 128
NUM_COLS = 44
#ENCODING_SIZE = 64
learning_rate_autoencoder = 1e-3

SEQ_LEN = pretrain_dataset.get_seq_len()
VOCAB_SIZE = pretrain_dataset.get_vocab_size()

num_epochs_ft = 40
learning_rate_ft = 5e-3

encoder = ExpEncoder(vocab_size=VOCAB_SIZE, sequence_len=SEQ_LEN, embedding_size=HIDDEN_SIZE, output_size=ENCODING_SIZE, num_cols=NUM_COLS).to(device)

rnn_model = GRUDecoder(
    input_size=ENCODING_SIZE,
    hidden_size=HIDDEN_SIZE,
    max_seq_len=14
).to(device)

# Define loss function and optimizer for RNN
ft_loss = nn.BCEWithLogitsLoss()
ft_optimizer = torch.optim.NAdam(list(rnn_model.parameters()) + list(encoder.parameters()) , lr=learning_rate_ft, weight_decay=1e-2, decoupled_weight_decay=True)
ft_scheduler = optim.lr_scheduler.CosineAnnealingLR(ft_optimizer, T_max = num_epochs_ft, eta_min = 1e-6, last_epoch = -1)

# Training loop
rnn_model.train()
encoder.train()
print("Ready!")

The model is going to set all input MASK to None
Ready!


In [21]:
loss_per_epoch = []
for epoch in range(num_epochs_ft):
    # print(epoch)
    loss_per_step = []
    loop_object  = tqdm(enumerate(train_dataloader), desc=f"Epochs {epoch}")
    for i, batch in loop_object :        
        ft_optimizer.zero_grad() 
        inputs, labels = batch
        labels = labels.to(torch.float).to(device)

        input_year, input_seq = inputs
        bs, ss = labels.size(0), input_year.size(1)
        input_year = input_year.reshape(-1).to(device)
        input_seq = input_seq.reshape(bs * ss, -1).to(device)

        encodings = encoder(input_year, input_seq).view(bs,ss, -1)
        mask = ((input_seq == 101).sum(-1) == NUM_COLS).view(bs,ss).detach()


        # Forward pass
        outputs = rnn_model(encodings, mask=~mask)

        loss = ft_loss(torch.flatten(outputs), labels)  
        loss_per_step.append(loss.detach().cpu().numpy())
        loop_object.set_postfix_str("mean loss: %.3f"%np.mean(loss_per_step[-100:]))

        #loss.backward(retain_graph=True)
        loss.backward()
        ft_optimizer.step()
    # On epoch end
    loss_per_epoch.append(np.mean(loss_per_step))
    ft_scheduler.step()

    print(f"Epoch {epoch+1}/{num_epochs_ft}, Loss: {loss_per_epoch[-1]:.4f}")
    

Epochs 0: 198it [00:23,  8.61it/s, mean loss: 0.500]


Epoch 1/40, Loss: 0.6215


Epochs 1: 198it [00:20,  9.74it/s, mean loss: 0.507]


Epoch 2/40, Loss: 0.5193


Epochs 2: 198it [00:20,  9.83it/s, mean loss: 0.529]


...

Epochs 13: 198it [00:21,  9.36it/s, mean loss: 0.499]


Epoch 14/40, Loss: 0.5113


Epochs 14: 198it [00:20,  9.64it/s, mean loss: 0.514]


Epoch 15/40, Loss: 0.5095


Epochs 15: 132it [00:13,  9.35it/s, mean loss: 0.548]

In [17]:
mask[0], input_seq[0:14], encodings

(tensor([False, False, False, False, False, False, False, False, False, False,
         False, False, False, False], device='mps:0'),
 tensor([[101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101,
          101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101,
          101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101,
          101, 101],
         [101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101,
          101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101,
          101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101,
          101, 101],
         [101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101,
          101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101,
          101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101,
          101, 101],
         [101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101,
   

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, average_precision_score

val_loss = []
preds = []
targets = []

## Set both models into the eval mode.=
rnn_model.eval()
autoencoder.eval()
for batch in test_dataloader:
    inputs, labels = batch
    labels = labels.to(torch.float).to(device)

    input_year, input_seq = inputs
    bs, ss = labels.size(0), 14
    input_year = input_year.reshape(-1).to(device)
    input_seq = input_seq.reshape(bs * ss, -1).to(device)

    encodings = autoencoder.get_encoding(input_year, input_seq).view(bs,ss, -1)
    survey_emb = aggregator(encodings)


    # Forward pass
    xx = rnn_model(survey_emb)
    outputs = torch.nn.functional.sigmoid(xx).flatten()
    loss = ft_loss(outputs, labels)  
    val_loss.append(loss.detach().cpu().numpy())
    preds.extend(outputs.detach().cpu().numpy().tolist())
    targets.extend(labels.cpu().numpy().tolist())

In [ ]:
survey_emb

tensor([[[  79.6868,  529.0562,  122.7735,  ...,   74.3379, -556.9098,
          -675.4937],
         [  86.1495,  570.1829,  132.3874,  ...,   80.2999, -599.9509,
          -728.2538],
         [  85.2354,  566.8043,  131.5119,  ...,   79.5485, -596.7887,
          -723.5583],
         ...,
         [  74.5646,  494.1296,  114.7229,  ...,   69.4697, -520.0018,
          -631.0103],
         [  71.2082,  481.3701,  111.2558,  ...,   66.8464, -507.5799,
          -613.6431],
         [  73.0651,  484.0807,  112.4068,  ...,   68.0308, -509.4069,
          -618.1808]],

        [[  79.6868,  529.0562,  122.7735,  ...,   74.3379, -556.9098,
          -675.4937],
         [  86.1495,  570.1829,  132.3874,  ...,   80.2999, -599.9509,
          -728.2538],
         [  85.2354,  566.8043,  131.5119,  ...,   79.5485, -596.7887,
          -723.5583],
         ...,
         [  70.9823,  480.2695,  110.9016,  ...,   66.5088, -506.5378,
          -612.0607],
         [  70.6130,  477.2684,  110.305

In [ ]:
# Concatenate all the batches
predictions = (torch.tensor(preds) > 0.5).float()
actuals = torch.tensor(targets).flatten()

# Calculate precision, recall, and F1 score
precision, recall, f1, _ = precision_recall_fscore_support(actuals.cpu().numpy(), predictions.cpu().numpy(), average='binary')
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Precision: 0.1364
Recall: 0.0600
F1 Score: 0.0833


In [ ]:
batch

[[tensor([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,  0],
          [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,  0],
          [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,  0],
          [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,  0],
          [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,  0],
          [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,  0]],
         device='mps:0'),
  tensor([[[101, 101, 101,  ..., 101, 101, 101],
           [101, 101, 101,  ..., 101, 101, 101],
           [101, 101, 101,  ..., 101, 101, 101],
           ...,
           [101, 101, 101,  ..., 101, 101, 101],
           [  0,  27,  34,  ..., 171,  76,  73],
           [101, 101, 101,  ..., 101, 101, 101]],
  
          [[101, 101, 101,  ..., 101, 101, 101],
           [101, 101, 101,  ..., 101, 101, 101],
           [101, 101, 101,  ..., 101, 101, 101],
           ...,
           [  0,  72, 101,  ..., 177,  30,  32],
           [  0, 101, 101,  ..., 177,  